In [3]:
import envPressureSubsEdges
import datetime
import numpy as np
import random
from scipy.special import softmax
from scipy import spatial
import pickle
from tensorboardX import SummaryWriter
from numpy import dot
from numpy.linalg import norm
import torch as th
from Net008_inference import *

In [4]:
scale = 9
probDeep = 1.0
predSteps = 3
saveFigures = False

In [ ]:
environ = envPressureSubsEdges.CornicheEnv(macrostep=10,predefined=False,scale=scale, GUI=True)

 Retrying in 1 seconds
Could not connect to TraCI server at localhost:55158 [Errno 61] Connection refused
 Retrying in 2 seconds
Could not connect to TraCI server at localhost:55158 [Errno 61] Connection refused
 Retrying in 3 seconds
Could not connect to TraCI server at localhost:55158 [Errno 61] Connection refused
 Retrying in 4 seconds


In [ ]:
tls = len(environ.action_space)

In [ ]:
obs = environ.reset()
cars,phaseobs = obs

In [ ]:
with open("amatrix_edges.pkl", 'rb') as f:
    A = pickle.load(f)
    
with open("stoppedEdges.pkl", 'rb') as f:
    stoppedEdges = pickle.load(f)
    
indices = {c:i for i,c in enumerate(list(A.columns))}

In [ ]:
net = th.load("Corniche_GATFlow_OneOmbedding_4steps_1epochs_10 s_Episode17_008_scale6_simplified.model").cuda().to(DEVICE)

In [ ]:
name = 'DeepPressure_008inf_%dsteps_INprediction_%1.2f_scale%d' % (predSteps,probDeep,scale)

writer = SummaryWriter(comment="Evaluation_Softmax_%s" % name)

In [ ]:
nids = th.LongTensor([x for x in range(g.number_of_nodes())])

def PredictOBS(obs):
    entered = torch.zeros((g.number_of_nodes(),1)).cuda().to(DEVICE)
    #consider the red all action
    freeEdges = 1 - np.sum([stoppedEdges[tl][len(stoppedEdges[tl])-1].values.squeeze() for tl in range(tls)], axis=0)
    freeEdges = th.FloatTensor(freeEdges).unsqueeze(1)        

    predicted,_, entered = net(g, nids.cuda().to(DEVICE), th.FloatTensor(obs).cuda().to(DEVICE).unsqueeze(1), freeEdges.cuda().to(DEVICE), entered)
    for i in range(predSteps-1):
        predicted,_, entered = net(g, nids.cuda().to(DEVICE), predicted, freeEdges.cuda().to(DEVICE), entered)
    
    return predicted

In [ ]:
def tlsPressure(INedges,OUTedges,current,prediction):
    global indices
    
    if random.random()>0.5:
        pIN = th.sum(prediction[[indices[i] for i in INedges]]).item()
    else:
        pIN = th.sum(current[[indices[i] for i in INedges]]).item()
    
    
    pOUT = th.sum(current[[indices[i] for i in OUTedges]]).item()
    
    return pIN - pOUT

In [ ]:
def getAllPressures(env,indices,counts):
    tlspressures = []
    
    prediction = PredictOBS(counts)
    counts = th.FloatTensor(counts)
    
    for tlsID in env.phases.getTls():
        pressures = []
        for phase in env.phases.getTlsPhases(tlsID):
            pressure = tlsPressure(env.edgesDict[tlsID][phase]["incoming"],env.edgesDict[tlsID][phase]["outgoing"],counts, prediction)
            pressures.append(pressure)
        tlspressures.append(pressures)
    return tlspressures

In [ ]:
with open("./%s" % name,'w') as fo:
    fo.write("exp_name\tstep\twaitingTime\tqueue\tco2\n")
    for step in range(6*60*24):
        pressures = getAllPressures(environ,indices,cars)
        actions = [ np.argmax(pressures[i]) for i in range(tls)]
        obs,_,_,measures = environ.step(actions)
        cars,phaseobs = obs
        if step%60==0:
            time = str(datetime.timedelta(seconds=step))
            #print("Completed trips at minute %s: %d" % (time,measures['completed_trips']))
            print("%d" % measures['waitingTime'])
        writer.add_scalar("waitingTime", measures['waitingTime'], step)
        writer.add_scalar("queue", measures['queue'], step)
        writer.add_scalar("co2", measures['co2'], step)
        if saveFigures:
            environ.conn.gui.screenshot('View #0','./screenshots/%s_%d.png' % (name,step))
            fo.write("%s\t%d\t%d\t%d\t%2f\n" % (name,step,measures['waitingTime'],measures['queue'],measures['co2']))

In [ ]:
pressures[0]

In [ ]:
pressures

In [ ]:
[indices[i] for i in [0,2,3]]

In [ ]:
random.random()

In [ ]:
stoppedEdges